In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
#from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
import optuna


C:\Users\pieta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_csv('cleaned_dataset.csv')

In [4]:
print(df.shape)
print(df.info())
print(df.describe())

(94891, 30)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94891 entries, 0 to 94890
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   bathroomcount      94891 non-null  int64  
 1   bedroomcount       94891 non-null  int64  
 2   constructionyear   94891 non-null  int64  
 3   country            94891 non-null  object 
 4   district           94891 non-null  object 
 5   fireplace          94891 non-null  int64  
 6   floodingzone       94891 non-null  object 
 7   furnished          94891 non-null  int64  
 8   garden             94891 non-null  int64  
 9   kitchen            94891 non-null  int64  
 10  livingarea         94891 non-null  float64
 11  locality           94888 non-null  object 
 12  monthlycharges     94891 non-null  float64
 13  numberoffacades    94891 non-null  int64  
 14  peb                94891 non-null  object 
 15  postalcode         94891 non-null  int64  
 16  price     

In [5]:
df.columns

Index(['bathroomcount', 'bedroomcount', 'constructionyear', 'country',
       'district', 'fireplace', 'floodingzone', 'furnished', 'garden',
       'kitchen', 'livingarea', 'locality', 'monthlycharges',
       'numberoffacades', 'peb', 'postalcode', 'price', 'propertyid',
       'province', 'region', 'roomcount', 'showercount', 'stateofbuilding',
       'subtypeofproperty', 'surfaceofplot', 'swimmingpool', 'terrace',
       'toiletcount', 'typeofproperty', 'typeofsale'],
      dtype='object')

refining cleaned data

In [13]:
import pandas as pd

def clean_data(df):
    # Drop column: 'country'
    df = df.drop(columns=['country'])
    # Drop column: 'propertyid'
    df = df.drop(columns=['propertyid'])
    # Filter rows based on column: 'constructionyear'
    df = df[df['constructionyear'] > 1950]
    # Filter rows based on column: 'constructionyear'
    df = df[df['constructionyear'] < 2025]
    # Replace missing values with the most common value of each column in: 'locality'
    df = df.fillna({'locality': df['locality'].mode()[0]})
    # Drop column: 'monthlycharges'
    df = df.drop(columns=['monthlycharges'])
    # Filter rows based on column: 'numberoffacades'
    df = df[df['numberoffacades'] < 5]
    # Drop column: 'postalcode'
    df = df.drop(columns=['postalcode'])
    # Filter rows based on column: 'roomcount'
    df = df[df['roomcount'] < 12]
    # Filter rows based on column: 'showercount'
    df = df[df['showercount'] < 5]
    # Filter rows based on column: 'stateofbuilding'
    df = df[df['stateofbuilding'] > 0]
    # Filter rows based on column: 'surfaceofplot'
    df = df[df['surfaceofplot'] < 20000]
    # Filter rows based on column: 'surfaceofplot'
    df = df[df['surfaceofplot'] <= 1000]
    # Filter rows based on column: 'toiletcount'
    df = df[df['toiletcount'] < 4]
    # Change column type to object for column: 'constructionyear'
    df = df.astype({'constructionyear': 'object'})
    return df

# Loaded variable 'df' from URI: c:\Users\pieta\OneDrive\Bureau\immoModeltraining\immoModel\cleaned_dataset.csv
df = pd.read_csv(r'c:\Users\pieta\OneDrive\Bureau\immoModeltraining\immoModel\cleaned_dataset.csv')

df_clean = clean_data(df.copy())
df_clean.head()

,bathroomcount,bedroomcount,constructionyear,district,fireplace,floodingzone,furnished,garden,kitchen,livingarea,...,roomcount,showercount,stateofbuilding,subtypeofproperty,surfaceofplot,swimmingpool,terrace,toiletcount,typeofproperty,typeofsale
0,1,1,1969,Brugge,0,NON_FLOOD_ZONE,0,0,1,29.0,...,1,0,4,flat_studio,203,0,1,1,2,residential_sale
2,2,4,2008,Brugge,0,NON_FLOOD_ZONE,1,0,1,111.0,...,3,0,4,house,0,0,0,2,1,residential_sale
3,1,4,1979,Veurne,0,NON_FLOOD_ZONE,0,1,1,113.6,...,9,1,2,house,170,0,1,2,1,residential_sale
4,0,2,1972,Hasselt,0,NON_FLOOD_ZONE,0,0,1,92.0,...,1,0,5,apartment,400,0,1,1,2,residential_sale
5,1,1,1994,Brussels,0,NON_FLOOD_ZONE,1,0,3,50.0,...,3,1,5,apartment,143,0,1,1,2,residential_sale


In [17]:
print(df_clean.columns)


Index(['bathroomcount', 'bedroomcount', 'constructionyear', 'district',
       'fireplace', 'floodingzone', 'furnished', 'garden', 'kitchen',
       'livingarea', 'locality', 'numberoffacades', 'peb', 'price', 'province',
       'region', 'roomcount', 'showercount', 'stateofbuilding',
       'subtypeofproperty', 'surfaceofplot', 'swimmingpool', 'terrace',
       'toiletcount', 'typeofproperty', 'typeofsale'],
      dtype='object')


Categorize str values
constructionyear, district, floodingzone, subtypeofproperty, typeofsale

In [24]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

columns_to_encode = ['constructionyear', 'district', 'floodingzone', 'subtypeofproperty', 'typeofsale', 'locality', 'peb','province', 'region']

data_to_encode = df_clean[columns_to_encode]

one = OneHotEncoder()

encoded_data = one.fit_transform(data_to_encode)

encoded_df = pd.DataFrame(encoded_data.toarray(), columns=one.get_feature_names_out(columns_to_encode))

df_final = pd.concat([df_clean.drop(columns=columns_to_encode), encoded_df], axis=1)

df_final.head(5)


,bathroomcount,bedroomcount,fireplace,furnished,garden,kitchen,livingarea,numberoffacades,price,roomcount,...,province_Hainaut,province_Limburg,province_Liège,province_Luxembourg,province_Namur,province_Walloon Brabant,province_West Flanders,region_Brussels,region_Flanders,region_Wallonie
0,1.0,1.0,0.0,0.0,0.0,1.0,29.0,2.0,99000.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,2.0,4.0,0.0,1.0,0.0,1.0,111.0,2.0,399000.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,1.0,4.0,0.0,0.0,1.0,1.0,113.6,2.0,230000.0,9.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,2.0,0.0,0.0,0.0,1.0,92.0,2.0,198000.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5,1.0,1.0,0.0,1.0,0.0,3.0,50.0,2.0,215000.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


Machine learning process